In [ ]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt2
from skfuzzy import control as ctrl


# Generate universe variables
# * Slipratio is on subjective ranges [0, 10] in unitless.
# *Acceleration is on subjective ranges [0, 10] in units of m/s^2.
# * Pressure force has a range of [0, 100] in units of N.

sr = np.arange(0, 11, 1)
acc = np.arange(0, 11, 1)
press = np.arange(0, 101, 1)
# Generate fuzzy membership functions
sr_z=fuzz.trimf(sr,[0,0,1])
sr_vs=fuzz.trimf(sr,[0.5,1.5,2.5])
sr_ts=fuzz.trimf(sr,[2,3,4])
sr_so=fuzz.trimf(sr,[3.5,4.5,5.5])
sr_o=fuzz.trimf(sr,[5,6,7])
sr_tl=fuzz.trimf(sr,[6.5,7.5,8.5])
sr_l=fuzz.trimf(sr,[8,10,10])
####################################################

a_nl=fuzz.trimf(acc,[0,0,2])
a_nm=fuzz.trimf(acc,[1,2.5,4])
a_ns=fuzz.trimf(acc,[3.5,5,6.5])
a_z=fuzz.trimf(acc,[6,7,8])
a_ps=fuzz.trimf(acc,[7.5,8.5,9.5])
a_pl=fuzz.trimf(acc,[8.5,10,10])


####################################################

p_pf=fuzz.trimf(press,[0, 16, 32])
p_ps=fuzz.trimf(press,[16, 32, 48])
p_z=fuzz.trimf(press,[32, 48, 64])
p_ns=fuzz.trimf(press,[48, 64, 80])
p_nf=fuzz.trimf(press,[64, 80, 96])


# Visualize these universes and membership functions
fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(10, 10))
ax0.plot(sr, sr_z, 'b', linewidth=1.5, label='ZERO')
ax0.plot(sr, sr_vs, 'g', linewidth=1.5, label='very small')
ax0.plot(sr, sr_ts, 'g', linewidth=1.5, label='too small')
ax0.plot(sr, sr_so, 'g', linewidth=1.5, label='smaller than optimal')
ax0.plot(sr, sr_o, 'g', linewidth=1.5, label='optimal')
ax0.plot(sr, sr_tl, 'g', linewidth=1.5, label='too large')
ax0.plot(sr, sr_l, 'g', linewidth=1.5, label='large')
ax0.set_title('slip ratio')
ax0.legend()

ax1.plot(acc, a_nl, 'b', linewidth=1.5, label='neg large')
ax1.plot(acc, a_nm, 'b', linewidth=1.5, label='neg medium')
ax1.plot(acc, a_ns, 'b', linewidth=1.5, label='neg small')
ax1.plot(acc, a_z, 'b', linewidth=1.5, label='zero')
ax1.plot(acc, a_ps, 'b', linewidth=1.5, label='pos small')
ax1.plot(acc, a_pl, 'b', linewidth=1.5, label='pos large')
ax1.set_title('Acceleration')
ax1.legend()


ax2.plot(press, p_pf, 'b', linewidth=1.5, label='positive fast')
ax2.plot(press, p_ps, 'b', linewidth=1.5, label='positive slow')
ax2.plot(press, p_z, 'b', linewidth=1.5, label=' zero')
ax2.plot(press, p_ns, 'b', linewidth=1.5, label='negative slow')
ax2.plot(press, p_nf, 'b', linewidth=1.5, label='negative fast')
ax2.set_title('Pressure amount')
ax2.legend()

# Turn off top/right axes
for ax in (ax0, ax1, ax2):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
plt.tight_layout()
plt.show()  # To show the membership inputs


sr = ctrl.Antecedent(sr, 'slip_ratio')
acc = ctrl.Antecedent(acc, 'acceleration')
press = ctrl.Consequent(press,'pressure')

####################################################

sr['z'] = fuzz.trimf(sr.universe, [0,0,1])
sr['vs'] = fuzz.trimf(sr.universe, [0.5,1.5,2.5])
sr['ts'] = fuzz.trimf(sr.universe, [2,3,4])
sr['so'] = fuzz.trimf(sr.universe, [3.5,4.5,5.5])
sr['o'] = fuzz.trimf(sr.universe, [5,6,7])
sr['tl'] = fuzz.trimf(sr.universe, [6.5,7.5,8.5])
sr['l'] = fuzz.trimf(sr.universe, [8,10,10])

####################################################



acc['nl'] = fuzz.trapmf(acc.universe, [0,0,2])
acc['nm'] = fuzz.trapmf(acc.universe, [1,2.5,4])
acc['ns'] = fuzz.trapmf(acc.universe, [3.5,5,6.5])
acc['z'] = fuzz.trapmf(acc.universe, [6,7,8])
acc['ps'] = fuzz.trapmf(acc.universe, [7.5,8.5,9.5])
acc['pl'] = fuzz.trapmf(acc.universe, [8.5,10,10])

####################################################


press['pf'] = fuzz.trimf(press.universe, [0, 16, 32])
press['ps'] = fuzz.trimf(press.universe, [16, 32, 48])
press['z'] = fuzz.trimf(press.universe, [32, 48, 64])
press['ns'] = fuzz.trimf(press.universe, [48, 64, 80])
press['nf'] = fuzz.trimf(press.universe, [64, 80, 96])

"""
The Fuzzy design rules
rule1= If zero & neg large= postive fast.
rule2= If  zero & neg medium = positive fast
rule3= If very samll & nega medium= positive fast
rule4= If very samll & nega small= positive small
rule5= If too samll & nega small= zero
rule6= If smaller than optimal & zero= zero
rule7= If smaller than optimal & positive small= zero
rule8= If optimum & positive small= negative slow
rule9= If  too large & positive small= negative slow
rule10= If  too large &positive large= negative fast
rule11= If large &positive large= negative fast

"""


rule1=ctrl.Rule(sr['z']&acc['nl'],press['pf'])
rule2=ctrl.Rule(sr['z']&acc['nm'],press['pf'])
rule3=ctrl.Rule(sr['vs']&acc['nm'],press['pf'])
rule4=ctrl.Rule(sr['vs']&acc['ns'],press['ps'])
rule5=ctrl.Rule(sr['ts']&acc['ns'],press['z'])
rule6=ctrl.Rule(sr['so']&acc['z'],press['z'])
rule7=ctrl.Rule(sr['so']&acc['ps'],press['z'])
rule8=ctrl.Rule(sr['o']&acc['ps'],press['ns'])
rule9=ctrl.Rule(sr['tl']&acc['ps'],press['ns'])
rule10=ctrl.Rule(sr['tl']&acc['pl'],press['nf'])
rule11=ctrl.Rule(sr['l']&acc['pl'],press['nf'])
  
press_ctrl= ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10, rule12])
pressure = ctrl.ControlSystemSimulation(press_ctrl)
pressure.input['slip ratio'] = 8
pressure.input['acceleration'] = 3

# Crunch the numbers
pressure.compute()
print (pressure.output['press'])
press.view(sim=pressure)
sim=ctrl.ControlSystemSimulation(press_ctrl, flush_after_run=24 * 24 + 1)
  
upsampled = np.linspace(0, 10, 25)
x, y = np.meshgrid(upsampled, upsampled)
z = np.zeros_like(x)
  
for i in range(10):
    for j in range(10):
        sim.input['sr'] = x[i, j]
        sim.input['acc'] = y[i, j]
        sim.compute()
        z[i, j] = sim.output['press']
  
  
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # Required for 3D plotting

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(x, y, z, rstride=1, cstride=1, cmap='viridis',

linewidth=0.4, antialiased=True)
cset = ax.contourf(x, y, z, zdir='z', offset=-2.5, cmap='viridis', alpha=0.5)
cset = ax.contourf(x, y, z, zdir='x', offset=3, cmap='viridis', alpha=0.5)
cset = ax.contourf(x, y, z, zdir='y', offset=3, cmap='viridis', alpha=0.5)
ax.view_init(10, 20)
plt.show()